imprements by https://www.kaggle.com/code/awsaf49/birdclef23-effnet-fsr-cutmixup-train

In [1]:
import sys
# ver 3
#sys.path.append('/kaggle/input/birdclef-2023-baseline')
# ver 4
#sys.path.append('/kaggle/input/baseline-add-gaussian')
# ver 6 
sys.path.append('/kaggle/input/exp6-baseline-holizontalflip')

sys.path.append('/kaggle/input/torchlibrosa-master')
sys.path.append('/kaggle/input/tinytag-master')

# Impoert library

In [2]:
import os
import random
from glob import glob
from pathlib import Path
import yaml
from tqdm import tqdm
import IPython.display as ipd

import numpy as np
import pandas as pd

import matplotlib as mpl
cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt

import librosa
import wandb
from sklearn import metrics

import torch
from torch import nn
from torch.nn import functional as F
from torch.cuda.amp import autocast, GradScaler

from timm.scheduler import CosineLRScheduler

In [3]:
from config import CFG
from pytorch_model import BirdCLEF23Net
import pytorch_modeler as modeler
import pytorch_preprocessing as prep
from pytorch_wav2logmel import Wav2Logmel
import common as com

In [4]:
modeler.set_seed(CFG.seed)
# setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print('Debug :', CFG.debug)

cuda:0
Debug : True


In [5]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを設定
jst = pytz.timezone('Asia/Tokyo')

# 現在時刻を取得し、日本時間に変換
now = datetime.now(jst)

# 現在時刻を文字列に変換
now_str = now.strftime('%Y-%m-%d %H:%M:%S')

print(now_str)

2023-05-04 22:28:54


# Data Frame

In [6]:
test_paths = glob('/kaggle/input/birdclef-2023/test_soundscapes/*ogg')
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df['filename'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.ogg',''))
test_df.head(2)

,filepath,filename
0,/kaggle/input/birdclef-2023/test_soundscapes/s...,soundscape_29201


# Inference

In [7]:
test_ds = prep.BirdDataset(test_df, is_train=False, is_test=True)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_ds,
    batch_size=CFG.test_bs,
    shuffle=False,
    num_workers = os.cpu_count(),
    collate_fn = modeler.inference_collate,
    )

In [8]:
inference_model_path = '/kaggle/working/multi_label/exp10_secondary_label/20230504_ver1_fold-0.pth'

In [9]:
num_classes = CFG.num_classes
   
# Compute batch size and number of samples to drop
infer_bs = CFG.test_bs
model = BirdCLEF23Net(num_classes=CFG.num_classes, pretrained=False)
    
# Test
print('# Loading model')
print(model.load_state_dict(torch.load(inference_model_path, map_location=device), strict=False))
model.to(device)
wav_to_logmel = Wav2Logmel().to(device)

# Predict on the validation data for oof result
print('# Infering Test')
model.eval()
test_pred = []
row_id = []
with torch.no_grad():
    for inputs, sample_info in tqdm(test_dataloader):
        inputs = inputs.to(device)
        logmel = wav_to_logmel(inputs)
        outputs = model(logmel)
        outputs = outputs['clipwise_output']
        test_pred.append(outputs.detach().cpu().numpy())
        row_id.extend(sample_info['row_id'])

test_pred = np.concatenate(test_pred, axis=0)

# Loading model
<All keys matched successfully>
# Infering Test


100%|██████████| 1/1 [00:03<00:00,  3.91s/it]


In [14]:
logmel.shape

torch.Size([120, 1, 224, 105])

In [10]:
bird_names = CFG.class_names
sub_cols = ['row_id'] + bird_names
sub_df = pd.DataFrame(columns=sub_cols)
sub_df['row_id'] = row_id
sub_df[bird_names] = test_pred

In [11]:
sub_df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.031590,0.023902,0.019060,0.008839,0.002298,0.013876,0.014658,0.003568,0.026662,...,0.009064,0.005359,0.011242,0.014829,0.005114,0.001724,0.009306,0.006735,0.012921,0.006005
1,soundscape_29201_10,0.007799,0.018663,0.013701,0.024924,0.001148,0.006868,0.005765,0.003631,0.006853,...,0.004489,0.004401,0.005877,0.007695,0.003406,0.003096,0.009559,0.005884,0.012262,0.009689
2,soundscape_29201_15,0.024862,0.034640,0.064203,0.033664,0.004775,0.021400,0.006689,0.003721,0.010968,...,0.010781,0.006842,0.028689,0.017682,0.006808,0.006595,0.017698,0.013018,0.024077,0.020706
3,soundscape_29201_20,0.013427,0.018761,0.018830,0.033060,0.002429,0.007490,0.009768,0.002667,0.021787,...,0.008764,0.005233,0.010698,0.012442,0.012263,0.003842,0.011248,0.008315,0.015213,0.008231
4,soundscape_29201_25,0.013861,0.019802,0.012881,0.051554,0.002975,0.017863,0.012681,0.007323,0.033791,...,0.013026,0.007187,0.015551,0.013286,0.016985,0.008153,0.014067,0.017962,0.045676,0.018149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.007281,0.012306,0.006136,0.051794,0.001498,0.008613,0.007138,0.003904,0.020537,...,0.005941,0.005450,0.008610,0.008247,0.006976,0.005487,0.010755,0.010908,0.028207,0.009174
116,soundscape_29201_585,0.004367,0.004605,0.001888,0.009920,0.000331,0.007695,0.004830,0.000414,0.006652,...,0.003436,0.001187,0.002654,0.001518,0.001046,0.001117,0.009461,0.001835,0.015365,0.003161
117,soundscape_29201_590,0.005605,0.011165,0.017578,0.038829,0.002629,0.014448,0.008960,0.002303,0.014126,...,0.007212,0.004149,0.006411,0.008491,0.005005,0.008784,0.013071,0.007377,0.034794,0.013178
118,soundscape_29201_595,0.008848,0.025153,0.007278,0.035170,0.003007,0.012693,0.008303,0.004449,0.025830,...,0.007291,0.005110,0.009211,0.010997,0.009355,0.006436,0.010702,0.008890,0.034741,0.013351


In [12]:
sub_df.to_csv('submission.csv',index=False)